In [1]:
# pip install h3

In [2]:
import os
import h3
import folium
import pandas as pd
import ast  # Para convertir la cadena de coordenadas a una tupla


In [3]:
# pip install pandas

In [4]:
# Función para obtener el índice H3 y las coordenadas
def get_h3_index_and_coords(latitude, longitude, resolution=7):
    h3_index = h3.geo_to_h3(latitude, longitude, resolution)
    coords = h3.h3_to_geo(h3_index)
    return h3_index, coords

repo = r'D:\trufiapp\GANS'
csv_file_path = os.path.join(repo, 'route_info.csv')
csv_save_file_path = os.path.join(repo, 'id_index_h3.csv')
output_dir = os.path.join(repo, 'cochabamba_data')  # Nuevo directorio para la data modificada
output_file_path = os.path.join(output_dir, 'cochabamba_data.csv')  # Ruta completa al archivo CSV


In [5]:

# Leer el archivo CSV original
df = pd.read_csv(csv_file_path, parse_dates=['date_time'], encoding='latin1')

# Filtrar registros que tienen 'Cochabamba' como ciudad de origen y destino
cochabamba_records = df[
    (df['origin_city'] == 'Cochabamba')
].copy()

# Crear columnas para índices H3 de origen y destino
cochabamba_records['origin_h3_index'], cochabamba_records['origin_coords'] = zip(*cochabamba_records.apply(lambda row: get_h3_index_and_coords(row['origin_latitude'], row['origin_longitude']), axis=1))
cochabamba_records['dest_h3_index'], cochabamba_records['dest_coords'] = zip(*cochabamba_records.apply(lambda row: get_h3_index_and_coords(row['destination_latitude'], row['destination_longitude']), axis=1))

# Crear un diccionario que mapea cada índice H3 único a su propio ID único
h3_index_to_id = {h3_index: idx + 1 for idx, h3_index in enumerate(pd.concat([cochabamba_records['origin_h3_index'], cochabamba_records['dest_h3_index']]).unique())}

# Crear DataFrame con índices H3 únicos y sus IDs correspondientes y coordenadas
h3_id_df = pd.DataFrame(list(h3_index_to_id.items()), columns=['h3_index', 'id'])
h3_id_df[['latitude', 'longitude']] = pd.DataFrame(h3_id_df['h3_index'].apply(lambda h3_index: h3.h3_to_geo(h3_index)).to_list(), index=h3_id_df.index)

# Guardar DataFrame de índices H3 y sus IDs en un archivo CSV separado
h3_id_df.to_csv(csv_save_file_path, index=False)

# Asignar los IDs únicos tanto para el origen como para el destino utilizando el diccionario
cochabamba_records['origin_id'] = cochabamba_records['origin_h3_index'].map(h3_index_to_id)
cochabamba_records['destination_id'] = cochabamba_records['dest_h3_index'].map(h3_index_to_id)

# Crear un nuevo DataFrame solo con las columnas necesarias
new_df = cochabamba_records[['date_time', 'hour', 'day_of_week', 'day_of_month', 'is_weekend', 'distance_meters', 'origin_id', 'destination_id']]

# Guardar el nuevo DataFrame con IDs únicos y solo registros en Cochabamba en un nuevo directorio
os.makedirs(output_dir, exist_ok=True)
new_df.to_csv(output_file_path, index=False)

print(f"La información modificada se ha guardado en el archivo {output_file_path}.")
max_origin_id = cochabamba_records['origin_id'].max()
max_destination_id = cochabamba_records['destination_id'].max()


# Obtener una lista de IDs únicos correspondientes a origin_id
unique_origin_ids = cochabamba_records['origin_id'].unique().tolist()

# Imprimir la lista de IDs únicos
print("Lista de IDs únicos de origin_id:")
print(unique_origin_ids)

print(f"El máximo ID de destinos en Cochabamba es: {max_destination_id}")


La información modificada se ha guardado en el archivo D:\trufiapp\GANS\cochabamba_data\cochabamba_data.csv.
Lista de IDs únicos de origin_id:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
El máximo ID de destinos en Cochabamba es: 497


In [6]:

repoCB = r'D:\trufiapp\GANS\cochabamba_data'
repo = r'D:\trufiapp\GANS'
csv_file_path = os.path.join(repoCB, 'cochabamba_data.csv')
csv_save_file_path = os.path.join(repo, 'id_index_h3.csv')

# Load the DataFrame with H3 indices and coordinates
h3_id_df = pd.read_csv(csv_save_file_path)

# Calculate the average latitude and longitude
avg_lat = h3_id_df['latitude'].mean()
avg_lon = h3_id_df['longitude'].mean()

# Create a map centered at the average coordinates
m = folium.Map(location=[avg_lat, avg_lon], zoom_start=12)

# Set to keep track of printed hexagons
printed_hexagons = set()

h3_id_df.head()

,h3_index,id,latitude,longitude
0,878b2c8a1ffffff,1,-17.397907,-66.169306
1,878b2c8aeffffff,2,-17.375229,-66.167926
2,878b2c8a3ffffff,3,-17.387033,-66.148877
3,878b2c8a0ffffff,4,-17.409708,-66.150256
4,878b2c98affffff,5,-17.455051,-66.153015


In [7]:

# Function to add a polygon to the map with a label
def add_polygon_with_label(m, hexagon, label):
    folium.Polygon(
        locations=hexagon,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.4,
        tooltip=f'Label: {label}'  # Add tooltip with the label number
    ).add_to(m)
    printed_hexagons.add(label)  # Use 'label' as a unique identifier

# Iterate over the records
for idx, row in h3_id_df.iterrows():
    # Get the coordinates and convert them to a tuple
    coords = (row['latitude'], row['longitude'])
    hexagon_coords = h3.h3_to_geo_boundary(row['h3_index'])  # Get hexagon coordinates

    # Check if the hexagon has not been printed yet
    if row['id'] not in printed_hexagons:  # Use 'id' as a unique identifier
        add_polygon_with_label(m, hexagon_coords, row['id'])  # Use 'id' as a label

# Save the map of origins and destinations in Cochabamba as an HTML file
map_cochabamba_file_path = os.path.join(repo, 'h3_map_cochabamba_.html')
m.save(map_cochabamba_file_path)


In [8]:

# Imprimir los hexágonos impresos
print("Hexágonos impresos:", len(printed_hexagons))

Hexágonos impresos: 497


In [9]:
# printed_hexagons

In [10]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\idaas\\anaconda3\\envs\\pyspark-env\\lib\\site-packages\\pyspark'

In [11]:
# Importar las bibliotecas necesarias
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import col, lit, concat_ws, collect_set
from pyspark.sql.types import TimestampType, StructField, StringType, IntegerType, StructType
from pyspark.sql.functions import date_format
sc = SparkContext.getOrCreate()
# Primera sesión de Spark
spark = SparkSession.builder \
    .appName("Trufi") \
    .config("spark.executor.memoryOverhead", "2g") \
    .config("spark.executor.memory", "3g") \
    .config("spark.driver.host", "localhost") \
    .config("spark.master", "local") \
    .getOrCreate()

# Configurar el número de particiones
spark.conf.set("spark.sql.shuffle.partitions", "4")
spark.sparkContext

<SparkContext master=local[*] appName=pyspark-shell>

In [12]:

# Define the path to your CSV file
csv_file_path = "D:\\trufiapp\\GANS\\cochabamba_data\\cochabamba_data.csv"

# Read the CSV file into a DataFrame
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Convert the timestamp to an hourly format
df = df.withColumn("Pickup_datetime_hourly", date_format(col("date_time").cast("timestamp"), "yyyy-MM-dd HH:00"))

# Create a trip count column
df = df.withColumn("Trip_count", lit(1))

# Group by pickup datetime, location ID, and aggregate the trip count
hourly_aggregated = df.groupby(['Pickup_datetime_hourly', 'origin_id']).agg({'Trip_count': 'count'}).withColumnRenamed("count(Trip_count)", "Trips_count")

# Crear la columna única usando concat_ws
hourly_aggregated = hourly_aggregated.select(
    concat_ws('_', col("Pickup_datetime_hourly"), col("origin_id")).alias("UniqueColumn"),
    col("Pickup_datetime_hourly"),
    col("origin_id"),
    col("Trips_count")
)

# Show the resulting DataFrame
hourly_aggregated.show(5)
hourly_aggregated.orderBy(col("Pickup_datetime_hourly").desc()).show(5)
# Count the number of rows in the aggregated DataFrame
print("Total rows in the aggregated DataFrame:", hourly_aggregated.count())


+------------------+----------------------+---------+-----------+
|      UniqueColumn|Pickup_datetime_hourly|origin_id|Trips_count|
+------------------+----------------------+---------+-----------+
|2022-09-12 09:00_1|      2022-09-12 09:00|        1|          1|
|2022-09-12 11:00_3|      2022-09-12 11:00|        3|          1|
|2022-09-12 19:00_5|      2022-09-12 19:00|        5|          3|
|2022-09-13 17:00_9|      2022-09-13 17:00|        9|          2|
|2022-09-13 18:00_3|      2022-09-13 18:00|        3|          7|
+------------------+----------------------+---------+-----------+
only showing top 5 rows

+-------------------+----------------------+---------+-----------+
|       UniqueColumn|Pickup_datetime_hourly|origin_id|Trips_count|
+-------------------+----------------------+---------+-----------+
| 2023-12-26 22:00_7|      2023-12-26 22:00|        7|          1|
| 2023-12-26 22:00_3|      2023-12-26 22:00|        3|         15|
|2023-12-26 22:00_15|      2023-12-26 22:00|  

In [13]:
# Tu DataFrame original
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)
df.show()

+-------------------+----+-----------+------------+----------+---------------+---------+--------------+
|          date_time|hour|day_of_week|day_of_month|is_weekend|distance_meters|origin_id|destination_id|
+-------------------+----+-----------+------------+----------+---------------+---------+--------------+
|2022-09-12 09:55:24|   9|          0|          12|         0|            934|        1|             2|
|2022-09-12 09:55:32|   9|          0|          12|         0|            998|        2|             3|
|2022-09-12 10:05:28|  10|          0|          12|         0|            998|        3|             2|
|2022-09-12 10:07:36|  10|          0|          12|         0|            797|        2|             3|
|2022-09-12 11:27:27|  11|          0|          12|         0|            797|        3|             2|
|2022-09-12 11:27:34|  11|          0|          12|         0|            797|        2|             3|
|2022-09-12 18:26:30|  18|          0|          12|         0|  

In [14]:
df = df.withColumn("OriginLocationID", col("origin_id").cast("string"))

In [15]:
# Reemplaza "df" con tu DataFrame actual y "origin_id_string" con el nombre de tu nueva columna
df = df.withColumn("UniqueColumn", concat_ws('_', df["date_time"], df["OriginLocationID"]))

# Seleccionar las columnas deseadas
df = df.select("UniqueColumn", "date_time", "OriginLocationID","distance_meters")

# Mostrar algunas filas para verificar
df.show()
df.count()

+--------------------+-------------------+----------------+---------------+
|        UniqueColumn|          date_time|OriginLocationID|distance_meters|
+--------------------+-------------------+----------------+---------------+
|2022-09-12 09:55:...|2022-09-12 09:55:24|               1|            934|
|2022-09-12 09:55:...|2022-09-12 09:55:32|               2|            998|
|2022-09-12 10:05:...|2022-09-12 10:05:28|               3|            998|
|2022-09-12 10:07:...|2022-09-12 10:07:36|               2|            797|
|2022-09-12 11:27:...|2022-09-12 11:27:27|               3|            797|
|2022-09-12 11:27:...|2022-09-12 11:27:34|               2|            797|
|2022-09-12 18:26:...|2022-09-12 18:26:30|               4|            959|
|2022-09-12 18:54:...|2022-09-12 18:54:27|               3|           1938|
|2022-09-12 18:54:...|2022-09-12 18:54:42|               3|           1938|
|2022-09-12 19:12:...|2022-09-12 19:12:20|               5|           3255|
|2022-09-12 

1361823

In [16]:
# Assuming df is your DataFrame
df = df[df['distance_meters'] >= 500]

# Obtener el número de filas y columnas después de aplicar el filtro
num_filas = df.count()
num_columnas = len(df.columns)

# Imprimir el número de filas y columnas del DataFrame filtrado
print("Número de filas después de filtrar:", num_filas)
print("Número de columnas:", num_columnas)


Número de filas después de filtrar: 1212411
Número de columnas: 4


In [17]:
import pandas as pd
from datetime import datetime, timedelta

def generate_series(start, stop, interval):
    start = pd.Timestamp(start)
    stop = pd.Timestamp(stop)
    timestamps = pd.date_range(start, stop, freq=f'{interval}s')
    return pd.DataFrame({'timestamp': timestamps})

# Generar el DataFrame de marcas de tiempo
timestamp_df = generate_series("2022-09-12", "2023-12-27", 3600)  # Intervalo de 1 hora

timestamp_df.tail()

,timestamp
11300,2023-12-26 20:00:00
11301,2023-12-26 21:00:00
11302,2023-12-26 22:00:00
11303,2023-12-26 23:00:00
11304,2023-12-27 00:00:00


In [18]:
# Generar el DataFrame de marcas de tiempo con intervalo de 1 hora
timestamp_df = pd.date_range("2022-09-12", "2023-12-27", freq='1H').to_frame(name='timestamp')

# Aplicar el formato de fecha deseado
timestamp_df['hourly_timestamp'] = timestamp_df['timestamp'].dt.strftime('%Y-%m-%d %H:00')
# Eliminar la última fila del DataFrame
timestamp_df = timestamp_df.iloc[:-1]

# Verificar la longitud después de la eliminación
len(timestamp_df)

11304

In [19]:
timestamp_df

,timestamp,hourly_timestamp
2022-09-12 00:00:00,2022-09-12 00:00:00,2022-09-12 00:00
2022-09-12 01:00:00,2022-09-12 01:00:00,2022-09-12 01:00
2022-09-12 02:00:00,2022-09-12 02:00:00,2022-09-12 02:00
2022-09-12 03:00:00,2022-09-12 03:00:00,2022-09-12 03:00
2022-09-12 04:00:00,2022-09-12 04:00:00,2022-09-12 04:00
...,...,...
2023-12-26 19:00:00,2023-12-26 19:00:00,2023-12-26 19:00
2023-12-26 20:00:00,2023-12-26 20:00:00,2023-12-26 20:00
2023-12-26 21:00:00,2023-12-26 21:00:00,2023-12-26 21:00
2023-12-26 22:00:00,2023-12-26 22:00:00,2023-12-26 22:00


In [20]:
# # Extrae los IDs de ubicación del conjunto de datos y guárdalos en una lista
# Pick_up_LocationID = df.select(collect_set('OriginLocationID').alias('locationID_list')).first()['locationID_list']
# # Convertir la lista de strings a números
# Pick_up_LocationID_numeric = list(map(int, Pick_up_LocationID))

# Pick_up_LocationID.sort()

# # Verificar el resultado
# print(Pick_up_LocationID_numeric)

In [21]:
from pyspark.sql.types import IntegerType
from pyspark.sql import Row

# Define la ruta al archivo id_index_h3.csv
id_index_file_path = "D:\\Trufiapp\\GANS\\id_index_h3.csv"

# Lee el archivo en un DataFrame de Spark
id_index_df = spark.read.csv(id_index_file_path, header=True, inferSchema=True)

# Encuentra los IDs de ubicación únicos del DataFrame original
Pick_up_LocationID = id_index_df.select('id').distinct()
Pick_up_LocationID = Pick_up_LocationID.orderBy("id")
Pick_up_LocationID = Pick_up_LocationID.withColumnRenamed("id", "OriginLocationID")
# Convierte la columna 'OriginLocationID' a tipo entero
Pick_up_LocationID = Pick_up_LocationID.withColumn("OriginLocationID", Pick_up_LocationID["OriginLocationID"].cast(IntegerType()))

# Convertir elementos de unique_origin_ids a enteros
unique_origin_ids_int = [int(id) for id in unique_origin_ids]

# Crear un DataFrame de Spark a partir de la lista unique_origin_ids_int
row_list = [Row(OriginLocationID=id) for id in unique_origin_ids_int]
schema = ["OriginLocationID"]
Pick_up_LocationID = spark.createDataFrame(row_list, schema=schema)

# Mostrar la cantidad de filas en el nuevo DataFrame
print("Cantidad de filas en Pick_up_LocationID:", Pick_up_LocationID.count())

Cantidad de filas en Pick_up_LocationID: 59


In [22]:
# Define una función para agregar la columna locationID al DataFrame de timestamp
def locations_timestamp_generator(dataframe, location_id):
    dataframe["OriginLocationID"] = location_id
    return dataframe

# Define el esquema para el DataFrame final
data_schema = [StructField("timestamp", TimestampType(), True),
               StructField("hourly_timestamp", StringType(), True),
               StructField("OriginLocationID", IntegerType(), True)]
final_struct = StructType(fields=data_schema)

In [23]:
# Crea un DataFrame vacío con las columnas requeridas
data_schema = ["timestamp", "hourly_timestamp", "OriginLocationID"]
ID_plus_timestamp = pd.DataFrame(columns=data_schema)

In [24]:
# Lista para almacenar DataFrames temporales
frames = []

# Itera sobre los IDs de ubicación y agrega las filas al DataFrame final
for row in Pick_up_LocationID.collect():
    location_id = row["OriginLocationID"]
    timestamp_df_copy = timestamp_df.copy()
    timestamp_df_copy = locations_timestamp_generator(timestamp_df_copy, location_id)
    frames.append(timestamp_df_copy)

# Concatena todos los DataFrames en la lista
ID_plus_timestamp = pd.concat(frames, ignore_index=True)

# Muestra el resultado
ID_plus_timestamp.count()

timestamp           666936
hourly_timestamp    666936
OriginLocationID    666936
dtype: int64

In [25]:

print("Total de filas en el DataFrame resultante:", len(ID_plus_timestamp))
print(ID_plus_timestamp.tail())

Total de filas en el DataFrame resultante: 666936
                 timestamp  hourly_timestamp  OriginLocationID
666931 2023-12-26 19:00:00  2023-12-26 19:00                59
666932 2023-12-26 20:00:00  2023-12-26 20:00                59
666933 2023-12-26 21:00:00  2023-12-26 21:00                59
666934 2023-12-26 22:00:00  2023-12-26 22:00                59
666935 2023-12-26 23:00:00  2023-12-26 23:00                59


In [26]:
# # Extraer los locationID únicos de ID_plus_timestamp
# location_ID_list2 = ID_plus_timestamp["OriginLocationID"].unique()

# # Verificar si hay algún locationID en ID_plus_timestamp que no exista en Pick_up_LocationID
# unique_list = [item for item in location_ID_list2 if item not in Pick_up_LocationID]

# # Debería estar vacío
# print("Lista de locationID únicos no encontrados en Pick_up_LocationID:", unique_list)

In [27]:
# Cambiar el locationID de entero a cadena
ID_plus_timestamp["locationID_string"] = ID_plus_timestamp["OriginLocationID"].astype(str)

# Crear la columna única utilizando concat_ws
ID_plus_timestamp["UniqueColumn_G"] = ID_plus_timestamp["hourly_timestamp"] + "_" + ID_plus_timestamp["locationID_string"]

# Seleccionar las columnas relevantes
ID_plus_timestamp = ID_plus_timestamp[["UniqueColumn_G", "timestamp", "hourly_timestamp", "OriginLocationID"]]

# Mostrar algunas filas para verificar
print(ID_plus_timestamp.head())
ID_plus_timestamp.count()

       UniqueColumn_G           timestamp  hourly_timestamp  OriginLocationID
0  2022-09-12 00:00_1 2022-09-12 00:00:00  2022-09-12 00:00                 1
1  2022-09-12 01:00_1 2022-09-12 01:00:00  2022-09-12 01:00                 1
2  2022-09-12 02:00_1 2022-09-12 02:00:00  2022-09-12 02:00                 1
3  2022-09-12 03:00_1 2022-09-12 03:00:00  2022-09-12 03:00                 1
4  2022-09-12 04:00_1 2022-09-12 04:00:00  2022-09-12 04:00                 1


UniqueColumn_G      666936
timestamp           666936
hourly_timestamp    666936
OriginLocationID    666936
dtype: int64

In [28]:
id_plus_timestamp = spark.createDataFrame(ID_plus_timestamp.copy())

In [29]:
hourly_aggregated.head()

Row(UniqueColumn='2022-09-12 09:00_1', Pickup_datetime_hourly='2022-09-12 09:00', origin_id=1, Trips_count=1)

In [30]:
from pyspark.sql.functions import col, lit, concat_ws
# Cambiar el ID de ubicación de entero a cadena
id_plus_timestamp = id_plus_timestamp.withColumn("locationID_string", col("OriginLocationID").cast("int"))

# Crear la columna única usando concat_ws
id_plus_timestamp = id_plus_timestamp.select(
    concat_ws('_', col("hourly_timestamp"), col("locationID_string")).alias("UniqueColumn_G"),
    col("timestamp"),
    col("hourly_timestamp"),
    col("OriginLocationID"),
    col("hourly_timestamp")
)

# Realizar la unión de ambos DataFrames usando la columna única
pick_aggregated = hourly_aggregated.join(id_plus_timestamp, hourly_aggregated["UniqueColumn"] == id_plus_timestamp["UniqueColumn_G"], "right")


In [31]:
# Mostrar algunas filas del resultado
# pick_aggregated.count()
pick_aggregated.head()

Row(UniqueColumn=None, Pickup_datetime_hourly=None, origin_id=None, Trips_count=None, UniqueColumn_G='2022-09-12 00:00_1', timestamp=datetime.datetime(2022, 9, 12, 0, 0), hourly_timestamp='2022-09-12 00:00', OriginLocationID=1, hourly_timestamp='2022-09-12 00:00')

In [32]:
# Seleccionar solo las columnas deseadas
columnas_deseadas = ['timestamp', 'hourly_timestamp', 'OriginLocationID', 'Trips_count', 'UniqueColumn_G']
pick_aggregated_o = pick_aggregated.select(*columnas_deseadas)

# Ordenar por las columnas en el orden deseado
pick_aggregated_o = pick_aggregated_o.orderBy(*[col(column) for column in columnas_deseadas])
# Reemplazar los valores nulos en las columnas especificadas
pick_aggregated_o = pick_aggregated_o.na.fill(0, "Trips_count")
pick_aggregated_o = pick_aggregated_o.na.fill(0, "OriginLocationID")

# Cambiar el nombre de Trips_count a origin_request_count
pick_aggregated_o = pick_aggregated_o.withColumnRenamed("Trips_count", "origin_request_count")

# Mostrar el DataFrame ordenado
pick_aggregated_o.show()

+-------------------+----------------+----------------+--------------------+-------------------+
|          timestamp|hourly_timestamp|OriginLocationID|origin_request_count|     UniqueColumn_G|
+-------------------+----------------+----------------+--------------------+-------------------+
|2022-09-12 00:00:00|2022-09-12 00:00|               1|                   0| 2022-09-12 00:00_1|
|2022-09-12 00:00:00|2022-09-12 00:00|               2|                   0| 2022-09-12 00:00_2|
|2022-09-12 00:00:00|2022-09-12 00:00|               3|                   0| 2022-09-12 00:00_3|
|2022-09-12 00:00:00|2022-09-12 00:00|               4|                   0| 2022-09-12 00:00_4|
|2022-09-12 00:00:00|2022-09-12 00:00|               5|                   0| 2022-09-12 00:00_5|
|2022-09-12 00:00:00|2022-09-12 00:00|               6|                   0| 2022-09-12 00:00_6|
|2022-09-12 00:00:00|2022-09-12 00:00|               7|                   0| 2022-09-12 00:00_7|
|2022-09-12 00:00:00|2022-09-1

In [33]:
import os
repo = r'D:\trufiapp\GANS'
out_agg_archivo_csv = os.path.join(repo, 'out_agg_archivo.csv')

In [34]:

# Especifica la ruta del archivo CSV en el sistema de archivos local
csv_local_path = "file:///" + out_agg_archivo_csv

# Guardar el DataFrame en formato CSV en el sistema de archivos local
pick_aggregated_o.write.mode("overwrite").option("header", "true").csv(csv_local_path)


In [35]:
# Lee el archivo CSV en un DataFrame
df = spark.read.csv("file:///" + out_agg_archivo_csv, header=True, inferSchema=True)

# Muestra el esquema del DataFrame
df.printSchema()

# Muestra las primeras filas del DataFrame
df.show()

root
 |-- timestamp: timestamp (nullable = true)
 |-- hourly_timestamp: timestamp (nullable = true)
 |-- OriginLocationID: integer (nullable = true)
 |-- origin_request_count: integer (nullable = true)
 |-- UniqueColumn_G: string (nullable = true)

+-------------------+-------------------+----------------+--------------------+-------------------+
|          timestamp|   hourly_timestamp|OriginLocationID|origin_request_count|     UniqueColumn_G|
+-------------------+-------------------+----------------+--------------------+-------------------+
|2023-01-05 08:00:00|2023-01-05 08:00:00|              32|                   0|2023-01-05 08:00_32|
|2023-01-05 08:00:00|2023-01-05 08:00:00|              33|                   0|2023-01-05 08:00_33|
|2023-01-05 08:00:00|2023-01-05 08:00:00|              34|                   0|2023-01-05 08:00_34|
|2023-01-05 08:00:00|2023-01-05 08:00:00|              35|                   0|2023-01-05 08:00_35|
|2023-01-05 08:00:00|2023-01-05 08:00:00|          

In [36]:
##AHORA PARA DESTINATIONS

In [37]:

# Define the path to your CSV file
csv_file_path = "D:\\trufiapp\\GANS\\cochabamba_data\\cochabamba_data.csv"

# Read the CSV file into a DataFrame
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Convert the timestamp to an hourly format
df = df.withColumn("Pickup_datetime_hourly", date_format(col("date_time").cast("timestamp"), "yyyy-MM-dd HH:00"))

# Create a trip count column
df = df.withColumn("Trip_count", lit(1))

# Group by pickup datetime, location ID, and aggregate the trip count
hourly_aggregated = df.groupby(['Pickup_datetime_hourly', 'destination_id']).agg({'Trip_count': 'count'}).withColumnRenamed("count(Trip_count)", "Trips_count")

# Crear la columna única usando concat_ws
hourly_aggregated = hourly_aggregated.select(
    concat_ws('_', col("Pickup_datetime_hourly"), col("destination_id")).alias("UniqueColumn"),
    col("Pickup_datetime_hourly"),
    col("destination_id"),
    col("Trips_count")
)

# Show the resulting DataFrame
hourly_aggregated.show()

# Count the number of rows in the aggregated DataFrame
print("Total rows in the aggregated DataFrame:", hourly_aggregated.count())


+-------------------+----------------------+--------------+-----------+
|       UniqueColumn|Pickup_datetime_hourly|destination_id|Trips_count|
+-------------------+----------------------+--------------+-----------+
| 2022-09-12 09:00_3|      2022-09-12 09:00|             3|          1|
| 2022-09-12 11:00_3|      2022-09-12 11:00|             3|          1|
| 2022-09-13 17:00_9|      2022-09-13 17:00|             9|          3|
|2022-09-13 18:00_61|      2022-09-13 18:00|            61|          1|
|2022-09-13 18:00_13|      2022-09-13 18:00|            13|          1|
| 2022-09-13 18:00_3|      2022-09-13 18:00|             3|          4|
|2022-09-13 18:00_12|      2022-09-13 18:00|            12|          1|
| 2022-09-13 19:00_9|      2022-09-13 19:00|             9|          2|
| 2022-09-13 19:00_2|      2022-09-13 19:00|             2|          1|
|2022-09-13 20:00_63|      2022-09-13 20:00|            63|          2|
| 2022-09-13 21:00_4|      2022-09-13 21:00|             4|     

In [38]:
# Realiza un left join de los DataFrames usando las columnas UniqueColumn y UniqueColumn_G
merged_df = pick_aggregated_o.join(hourly_aggregated, pick_aggregated_o["UniqueColumn_G"] == hourly_aggregated["UniqueColumn"], "left_outer")

# Selecciona las columnas deseadas en el DataFrame final
final_merged = merged_df.select(
    pick_aggregated_o["UniqueColumn_G"],
    pick_aggregated_o["hourly_timestamp"],
    pick_aggregated_o["OriginLocationID"],
    hourly_aggregated["Trips_count"].alias("destination_request_count"),
    pick_aggregated_o["origin_request_count"]
)

# Muestra el DataFrame final
final_merged.show()
final_merged.count()

+-------------------+----------------+----------------+-------------------------+--------------------+
|     UniqueColumn_G|hourly_timestamp|OriginLocationID|destination_request_count|origin_request_count|
+-------------------+----------------+----------------+-------------------------+--------------------+
| 2022-09-12 00:00_1|2022-09-12 00:00|               1|                     null|                   0|
| 2022-09-12 03:00_1|2022-09-12 03:00|               1|                     null|                   0|
| 2022-09-12 08:00_1|2022-09-12 08:00|               1|                     null|                   0|
| 2022-09-12 10:00_1|2022-09-12 10:00|               1|                     null|                   0|
| 2022-09-12 11:00_1|2022-09-12 11:00|               1|                     null|                   0|
| 2022-09-12 12:00_1|2022-09-12 12:00|               1|                     null|                   0|
| 2022-09-12 13:00_1|2022-09-12 13:00|               1|                  

666936

In [39]:
final_merged = final_merged.na.fill(0, "destination_request_count")

In [40]:
final_merged.orderBy(col("hourly_timestamp").desc()).show(10)

# Mostrar el DataFrame ordenado
# final_merged.show(1000)

+-------------------+----------------+----------------+-------------------------+--------------------+
|     UniqueColumn_G|hourly_timestamp|OriginLocationID|destination_request_count|origin_request_count|
+-------------------+----------------+----------------+-------------------------+--------------------+
|2023-12-26 23:00_15|2023-12-26 23:00|              15|                        0|                   0|
|2023-12-26 23:00_13|2023-12-26 23:00|              13|                        0|                   0|
|2023-12-26 23:00_14|2023-12-26 23:00|              14|                        0|                   0|
|2023-12-26 23:00_48|2023-12-26 23:00|              48|                        0|                   0|
| 2023-12-26 23:00_1|2023-12-26 23:00|               1|                        0|                   0|
| 2023-12-26 23:00_2|2023-12-26 23:00|               2|                        0|                   0|
|2023-12-26 23:00_57|2023-12-26 23:00|              57|                  

In [41]:
from pyspark.sql.functions import year, month, dayofmonth, hour, dayofweek
from pyspark.sql.functions import when

# Create Year column
final_merged = final_merged.withColumn('Year', year(final_merged['hourly_timestamp']))

# Create Month column
final_merged = final_merged.withColumn('Month', month(final_merged['hourly_timestamp']))

# Create Day of Month column
final_merged = final_merged.withColumn('DayOfMonth', dayofmonth(final_merged['hourly_timestamp']))

# Create Hour column
final_merged = final_merged.withColumn('Hour', hour(final_merged['hourly_timestamp']))

# Create Day of Week column
final_merged = final_merged.withColumn("DayOfWeek", dayofweek(col("hourly_timestamp")))

# Crear la columna isWeekend
final_merged = final_merged.withColumn("isWeekend", when((col("DayOfWeek") == 1) | (col("DayOfWeek") == 7), 1).otherwise(0))

# Show the resulting DataFrame
final_merged.count()


666936

In [42]:

# Especifica la ruta del archivo CSV en el sistema de archivos local
csv_local_path = "file:///" + out_agg_archivo_csv

# Guardar el DataFrame en formato CSV en el sistema de archivos local
final_merged.write.mode("overwrite").option("header", "true").csv(csv_local_path)


In [43]:
spark.stop()

In [44]:
# # Obtener la lista de ubicaciones (zonas)
# location_ids = df.select(collect_set('origin_id').alias('locationID_list')).first()['locationID_list']

# # Generar un DataFrame con todas las combinaciones de timestamp y ubicaciones
# def generate_series(start, stop, interval, location_ids):
#     start, stop = spark.createDataFrame([(start, stop)], ("start", "stop")).select(
#         [col(c).cast("timestamp").cast("long") for c in ("start", "stop")]
#     ).first()
    
#     timestamp_df = spark.range(start, stop, interval).select(
#         col("id").cast("timestamp").alias("timestamp")
#     )
    
#     timestamp_df = timestamp_df.withColumn("hourly_timestamp", date_format(col("timestamp").cast("timestamp"), "yyyy-MM-dd HH:00"))

#     # Crear el DataFrame con todas las combinaciones de timestamp y ubicaciones
#     data_schema = [StructField("timestamp", TimestampType(), True),
#                    StructField("hourly_timestamp", StringType(), True),
#                    StructField("locationID", IntegerType(), True)]
#     final_struct = StructType(fields=data_schema)
    
#     timestamp_location_df = spark.createDataFrame([], schema=final_struct)

#     # Iterar sobre las ubicaciones y agregarlas al DataFrame
#     for location_id in location_ids:
#         dataframe = timestamp_df.withColumn("locationID", lit(location_id))
#         timestamp_location_df = timestamp_location_df.union(dataframe)

#     return timestamp_location_df

# # Generar el DataFrame de timestamp y ubicaciones
# timestamp_location_df = generate_series("2022-09-12", "2022-10-08", 60 * 60, location_ids)

# # Convertir la ubicación a cadena y crear la columna única
# timestamp_location_df = timestamp_location_df.withColumn("UniqueColumn_G", concat_ws('_', timestamp_location_df.hourly_timestamp,
#                                                            col("locationID").cast(StringType())))
# timestamp_location_df = timestamp_location_df.drop("locationID_string")  # Eliminar la columna innecesaria

# # Mostrar algunas filas para verificar
# timestamp_location_df.show(5)

# # Unir el DataFrame generado con tus datos originales
# df_filled = timestamp_location_df.join(df, "UniqueColumn_G", "left_outer")

# # Mostrar el DataFrame resultante
# df_filled.show()

# # Contar el número de filas
# print("Total de filas en el DataFrame resultante:", df_filled.count())

In [45]:
# 

In [46]:
# #TODOS LOS POLIGONOS

# import os
# import h3
# import folium
# import pandas as pd

# repo = r'D:\trufiapp\GANS'
# # csv_file_path = os.path.join(repo, 'route_info.csv')
# # csv_save_file_path = os.path.join(repo, 'id_index_h3.csv')

# # Cargar datos del archivo CSV en un DataFrame de Pandas
# df = pd.read_csv(csv_file_path, encoding='latin-1')

# # Cargar datos del archivo CSV con índices H3 en un DataFrame de Pandas
# h3_id_df = pd.read_csv(csv_save_file_path)

# # Combinar los DataFrames en base a los índices H3 para origen
# df_combined_origin = pd.merge(df, h3_id_df, how='inner', left_on='origin_h3_index', right_on='h3_index')

# # Crear un mapa con folium
# m = folium.Map(location=[df_combined_origin['origin_latitude'].mean(), df_combined_origin['origin_longitude'].mean()], zoom_start=12)

# # Conjunto para realizar un seguimiento de hexágonos impresos
# printed_hexagons = set()

# # Función para agregar polígono al mapa con etiqueta
# def add_polygon_with_label(m, hexagon, label):
#     folium.Polygon(
#         locations=hexagon,
#         color='blue',
#         fill=True,
#         fill_color='blue',
#         fill_opacity=0.4,
#         tooltip=f'Label: {label}'  # Añadir tooltip con el número de etiqueta
#     ).add_to(m)

# # Iterar sobre los registros
# for idx, row in df_combined_origin.iterrows():
#     hexagon_origin = h3.h3_to_geo_boundary(row['origin_h3_index'])
    
#     # Verificar si el hexágono ya ha sido impreso
#     if row['origin_h3_index'] not in printed_hexagons:
#         add_polygon_with_label(m, hexagon_origin, row['id'])  # Utilizar 'id' como etiqueta
#         printed_hexagons.add(row['origin_h3_index'])  # Agregar el hexágono al conjunto

# # Guardar el mapa de origenes y destinos como un archivo HTML
# map_file_path = os.path.join(repo, 'h3_map_all.html')
# m.save(map_file_path)
